## Parte 3 - Classificação - Algoritmos

---
<img align="center" width="150" src="https://logodownload.org/wp-content/uploads/2017/05/ifood-logo-0.png">



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install scikit-learn==1.4.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 78.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0


In [3]:
!pip install feature-engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.0/375.0 kB 5.6 MB/s eta 0:00:00


In [4]:
import os
import numpy as np
import pandas as pd

## Lendos os Dados

In [5]:
df_abt_train = pd.read_csv('/content/drive/MyDrive/FIA Analytics/ML/4 - Técnicas de Classificação/Aprendizagem Supervisionada – Classificação _ Banco de Dados/Ifood/abt_classificacao_ifood_train.csv')
df_abt_train.head(2)

,data_ref,customer_id,uf,receita_1m,qtd_pedidos_1m,recencia,churn_next_month
0,2019-07-01 00:00:00+00:00,0001a8e61d8b08ad436e8e6f4adeb399b88df962c72d9d...,PR,102.0,2,15,1
1,2019-07-01 00:00:00+00:00,0001a9f97d01d2696cf70c7657ee2d039388d691720ff9...,SP,52.0,1,19,1


In [6]:
df_abt_oot = pd.read_csv('/content/drive/MyDrive/FIA Analytics/ML/4 - Técnicas de Classificação/Aprendizagem Supervisionada – Classificação _ Banco de Dados/Ifood/abt_classificacao_ifood_oot.csv')
df_abt_oot.head(2)

,data_ref,customer_id,uf,receita_1m,qtd_pedidos_1m,recencia,churn_next_month
0,2019-08-01 00:00:00+00:00,000a1fac4f7a67cc3f2e7667167597cb2c9a1b9edafe18...,SP,85.89,3,11,0
1,2019-08-01 00:00:00+00:00,0014b7013c66a05d0b5ce0687d614ac220d3ae1af398d2...,PE,211.96,5,11,0


In [7]:
key_vars = ['data_ref', 'customer_id']
num_vars = ['receita_1m', 'qtd_pedidos_1m', 'recencia']
cat_vars = ['uf']
target = 'churn_next_month'
features = cat_vars + num_vars

In [8]:
# Dados de treinamento somente com as variáveis numéricas
X_train = df_abt_train[features]
y_train = df_abt_train[target]

# Dados de avaliação (out of time) somente com as variáveis numéricas
X_oot = df_abt_oot[features]
y_oot = df_abt_oot[target]

In [9]:
print(X_train.shape, y_train.shape)

(30079, 4) (30079,)


In [10]:
print(X_oot.shape, y_oot.shape)

(18441, 4) (18441,)


**`Q1`** Treine uma Random Forest na base de treino usando Stratified K-Fold e avalie utilizando a métrica **AUC** na base Out of Time.

- Utilize o parametro `random_state=42` no algoritmo.
- Utilize os seguintes parâmetros `n_splits=5, shuffle=True, random_state=42` para Stratified K-Fold.
- Lembre-se de realizar o pré-processamento dos dados

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from feature_engine.wrappers import SklearnTransformerWrapper
from feature_engine.imputation import ArbitraryNumberImputer, CategoricalImputer
from feature_engine.encoding import OneHotEncoder

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
rf_pipe = Pipeline(steps=[
    ("numeric_imputer", ArbitraryNumberImputer(variables=num_vars,arbitrary_number=-999)),
    ("categorig_imputer",CategoricalImputer(variables=cat_vars,fill_value="missing")),
    ("ohe", OneHotEncoder(variables=cat_vars)),
    ("rf",RandomForestClassifier(random_state=42))
    ])

In [17]:
from sklearn.model_selection import StratifiedKFold, cross_validate

In [18]:
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)

In [22]:
cv_rf = cross_validate(rf_pipe,X_train,y_train,scoring=["accuracy","precision","recall","f1","roc_auc"],cv=skf,n_jobs=-1)

In [23]:
cv_rf = pd.DataFrame(cv_rf)
cv_rf.mean()

,0
fit_time,7.412956
score_time,0.723194
test_accuracy,0.637189
test_precision,0.533140
test_recall,0.500689
test_f1,0.516361
test_roc_auc,0.687022


In [24]:
cv_dt = cross_validate(rf_pipe,X_train,y_train,scoring=["accuracy","precision","recall","f1","roc_auc"],cv=skf,n_jobs=-1)
cv_dt = pd.DataFrame(cv_rf)
cv_dt.mean()

,0
fit_time,7.412956
score_time,0.723194
test_accuracy,0.637189
test_precision,0.533140
test_recall,0.500689
test_f1,0.516361
test_roc_auc,0.687022


In [67]:
rf_pipe.fit(X_train,y_train)

Pipeline(steps=[('numeric_imputer',
                 ArbitraryNumberImputer(arbitrary_number=-999,
                                        variables=['receita_1m',
                                                   'qtd_pedidos_1m',
                                                   'recencia'])),
                ('categorig_imputer',
                 CategoricalImputer(fill_value='missing', variables=['uf'])),
                ('ohe', OneHotEncoder(variables=['uf'])),
                ('rf', RandomForestClassifier(random_state=42))])

In [68]:
predicoes_rf = rf_pipe.predict(X_oot)
predicoes_rf

array([0, 0, 0, ..., 0, 1, 0])

In [69]:
predicoes_proba_rf = rf_pipe.predict_proba(X_oot)[:,1]
predicoes_proba_rf


array([0.05      , 0.22      , 0.33      , ..., 0.45533333, 0.84      ,
       0.06      ])

In [70]:
roc_rf = roc_auc_score(y_oot, predicoes_proba_rf)
roc_rf


0.6804526243869495

**`Q2`** Treine um LGBM na base de treino usando Stratified K-Fold e avalie utilizando a métrica **AUC** na base Out of Time.

- Utilize o parametro `random_state=42` no algoritmo.
- Utilize os seguintes parâmetros `n_splits=5, shuffle=True, random_state=42` para Stratified K-Fold.
- Lembre-se de realizar o pré-processamento dos dados

In [25]:
!pip install lightgbm

In [27]:
from lightgbm import LGBMClassifier

In [28]:
from sklearn.metrics import roc_auc_score

In [29]:
lg_pipe = Pipeline(steps=[
    ("numeric_imputer", ArbitraryNumberImputer(variables=num_vars,arbitrary_number=-999)),
    ("categorig_imputer",CategoricalImputer(variables=cat_vars,fill_value="missing")),
    ("ohe", OneHotEncoder(variables=cat_vars)),
    ("lg",LGBMClassifier(random_state=42))
    ])

In [33]:
from sklearn.model_selection import StratifiedKFold, cross_validate

In [34]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [35]:
cv_lg = cross_validate(lg_pipe, X_train, y_train, scoring=["accuracy", "precision", "recall", "f1", "roc_auc"], cv=skf, n_jobs=-1)
cv_lg = pd.DataFrame(cv_lg)
cv_lg.mean()

,0
fit_time,0.973623
score_time,0.419892
test_accuracy,0.691346
test_precision,0.594300
test_recall,0.637311
test_f1,0.615015
test_roc_auc,0.752186


In [36]:
cv_lg

,fit_time,score_time,test_accuracy,test_precision,test_recall,test_f1,test_roc_auc
0,1.113873,0.436868,0.689827,0.593371,0.630584,0.611412,0.752130
1,0.964452,0.499365,0.693318,0.598773,0.628866,0.613451,0.752893
2,0.972245,0.427091,0.683511,0.585072,0.626289,0.604979,0.745579
3,1.103319,0.507617,0.697307,0.598674,0.659648,0.627684,0.756710
4,0.714228,0.228522,0.692768,0.595609,0.641169,0.617550,0.753618


In [59]:
lg_pipe.fit(X_train,y_train)

[LightGBM] [Info] Number of positive: 11638, number of negative: 18441
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002251 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 373
[LightGBM] [Info] Number of data points in the train set: 30079, number of used features: 28
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.386914 -> initscore=-0.460301
[LightGBM] [Info] Start training from score -0.460301


Pipeline(steps=[('numeric_imputer',
                 ArbitraryNumberImputer(arbitrary_number=-999,
                                        variables=['receita_1m',
                                                   'qtd_pedidos_1m',
                                                   'recencia'])),
                ('categorig_imputer',
                 CategoricalImputer(fill_value='missing', variables=['uf'])),
                ('ohe', OneHotEncoder(variables=['uf'])),
                ('lg', LGBMClassifier(random_state=42))])

In [63]:
from sklearn.metrics import roc_auc_score

In [60]:
predicoes_lg = lg_pipe.predict(X_oot)
predicoes_lg

array([0, 0, 0, ..., 1, 0, 0])

In [64]:
predicoes_proba_lg = lg_pipe.predict_proba(X_oot)[:,1]
predicoes_proba_lg

array([0.25715269, 0.16641995, 0.36038471, ..., 0.57525861, 0.30179387,
       0.24529614])

In [65]:
roc_lg = roc_auc_score(y_oot, predicoes_proba_lg)
roc_lg

0.7332745686567761

**`Q3`** Treine um CatBoost na base de treino usando Stratified K-Fold e avalie utilizando a métrica **AUC** na base Out of Time.



In [37]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.3 MB/s eta 0:00:00


In [38]:
from catboost import CatBoostClassifier

In [48]:
from sklearn.metrics import roc_auc_score

In [49]:
cat_pipe = Pipeline(steps=[
    ("numeric_imputer", ArbitraryNumberImputer(variables=num_vars,arbitrary_number=-999)),
    ("categorig_imputer",CategoricalImputer(variables=cat_vars,fill_value="missing")),
    ("ohe", OneHotEncoder(variables=cat_vars)),
    ("cat",CatBoostClassifier(random_state=42))
    ])


In [50]:
from sklearn.model_selection import StratifiedKFold, cross_validate

In [51]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [52]:
cv_cat = cross_validate(cat_pipe, X_train, y_train, scoring=["accuracy", "precision", "recall", "f1", "roc_auc"], cv=skf, n_jobs=-1)
cv_cat = pd.DataFrame(cv_cat)
cv_cat.mean()

,0
fit_time,21.950994
score_time,0.323482
test_accuracy,0.691014
test_precision,0.595904
test_recall,0.625711
test_f1,0.610411
test_roc_auc,0.752546


In [53]:
cat_pipe.fit(X_train,y_train)

Learning rate set to 0.044071
0:	learn: 0.6805587	total: 56.1ms	remaining: 56.1s
1:	learn: 0.6694912	total: 66.2ms	remaining: 33s
2:	learn: 0.6595210	total: 74.1ms	remaining: 24.6s
3:	learn: 0.6501366	total: 82.1ms	remaining: 20.5s
4:	learn: 0.6419883	total: 89.9ms	remaining: 17.9s
5:	learn: 0.6345792	total: 97.7ms	remaining: 16.2s
6:	learn: 0.6279115	total: 106ms	remaining: 15s
7:	learn: 0.6216539	total: 113ms	remaining: 14s
8:	learn: 0.6164232	total: 121ms	remaining: 13.3s
9:	learn: 0.6113737	total: 128ms	remaining: 12.7s
10:	learn: 0.6066090	total: 136ms	remaining: 12.2s
11:	learn: 0.6028462	total: 143ms	remaining: 11.8s
12:	learn: 0.5988335	total: 152ms	remaining: 11.5s
13:	learn: 0.5952078	total: 159ms	remaining: 11.2s
14:	learn: 0.5920015	total: 167ms	remaining: 11s
15:	learn: 0.5890553	total: 175ms	remaining: 10.8s
16:	learn: 0.5865849	total: 184ms	remaining: 10.6s
17:	learn: 0.5843112	total: 197ms	remaining: 10.7s
18:	learn: 0.5821240	total: 206ms	remaining: 10.6s
19:	learn: 0.

Pipeline(steps=[('numeric_imputer',
                 ArbitraryNumberImputer(arbitrary_number=-999,
                                        variables=['receita_1m',
                                                   'qtd_pedidos_1m',
                                                   'recencia'])),
                ('categorig_imputer',
                 CategoricalImputer(fill_value='missing', variables=['uf'])),
                ('ohe', OneHotEncoder(variables=['uf'])),
                ('cat',
                 <catboost.core.CatBoostClassifier object at 0x7fa334fb9810>)])

In [54]:
predicoes_cat = cat_pipe.predict(X_oot)
predicoes_cat

array([0, 0, 0, ..., 1, 0, 0])

In [55]:
predicoes_proba_cat = cat_pipe.predict_proba(X_oot)[:,1]
predicoes_proba_cat

array([0.2721919 , 0.1516915 , 0.38197764, ..., 0.58245757, 0.28686931,
       0.24197551])

In [56]:
roc_cat = roc_auc_score(y_oot, predicoes_proba_cat)
roc_cat

0.7335190176031258